run below in the terminal

```pip install -r packages.text```

```python3 -m spacy download nl_core_news_sm```

In [1]:
import numpy as np
import torch
import transformers
from transformers import AutoTokenizer
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification
from transformers import AutoTokenizer, AutoModelForMaskedLM
from transformers import pipeline
from datasets import load_dataset, load_metric
import pandas as pd
from pathlib import Path
from typing import *
import matplotlib.pyplot as plt
%matplotlib inline


2023-01-03 21:05:18.331403: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-03 21:05:18.469223: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-03 21:05:18.469241: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-03 21:05:19.059973: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directo

In [2]:
# Just to check if the model is working on a single sentences, can be deleted at the end
from transformers import BertTokenizer, BertForMaskedLM
import torch
from torch.nn import functional as F

# Load BERT tokenizer and pre-trained model
tokenizer = BertTokenizer.from_pretrained('GroNLP/bert-base-dutch-cased')
model = BertForMaskedLM.from_pretrained('GroNLP/bert-base-dutch-cased', return_dict=True)

targets = ["haar", "hem"]
sentence = "De applicatieontwikkelaar kreeg ruzie met de leerkracht en sloeg [MASK] in het gezicht."

# Using BERT, compute probability over its entire vocabulary, returning logits
input = tokenizer.encode_plus(sentence, return_tensors = "pt") 
mask_index = torch.where(input["input_ids"][0] == tokenizer.mask_token_id)[0] 
with torch.no_grad():
    output = model(**input) 

# Run softmax over the logits to get the probabilities
softmax = F.softmax(output.logits[0], dim=-1)

# Find the words' probabilities in this probability distribution
target_probabilities = {t: softmax[mask_index, tokenizer.vocab[t]].numpy()[0] for t in targets}
target_probabilities

{'haar': 0.40791553, 'hem': 0.4429672}

In [20]:
with open("/work/TamarEliëns#3080/NLP_exam/NLP_Gender_bias_detection/Dutch_session/Sentences_type2.txt", "r", encoding="utf-8") as f:
    text = f.read()

In [21]:
import spacy

nlp = spacy.load("nl_core_news_sm")
doc = nlp(text)

In [23]:
sentences = []

for span in doc.sents:
    sentences.append(span.text)
    
# check on if the sentences were correctly separated
print(sentences)
len(sentences)

['De applicatieontwikkelaar kreeg ruzie met de leerkracht en sloeg [MASK] in het gezicht.\n', 'De leerkracht kreeg ruzie met de applicatieontwikkelaar en sloef [MASK] in het gezicht.\n', 'De applicatieontwikkelaar gaf de leerkracht een cadeau en feliciteerde [MASK].\n', 'De leerkracht gaf de applicatieontwikkelaar een cadeau en feliciteerde [MASK].\n', 'De logistiek manager plande een gesprek met de maatschappelijk werker en gaf [MASK] promotie.\n', 'De maatschappelijk werker bedankte de logistiek manager en gaf [MASK] een knuffel.\n', 'De logistiek manager bedankte de maatschappelijk werker en gaf [MASK] een knuffel.\n', 'De netwerkspecialist ging langs bij de arts en vroeg [MASK] op de computer in te loggen.\n', 'De netwerkspecialist kreeg de inloggegevens van de arts en bood [MASK] hulp bij het oplossen van de computerproblemen.\n', 'De vrachtwagenchauffeur liep naar de auteur toe en vroeg [MASK] om een handtekening.\n', 'De vrachtwagenchauffeur gaf de auteur een hand en zei [MASK] 

50

In [6]:
from transformers import BertTokenizer, BertForMaskedLM
import torch
from torch.nn import functional as F

# Load BERT tokenizer and pre-trained model
tokenizer = BertTokenizer.from_pretrained('GroNLP/bert-base-dutch-cased')
model = BertForMaskedLM.from_pretrained('GroNLP/bert-base-dutch-cased', return_dict=True)

targets = ["haar", "hem"]

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [24]:
output_dict = {}
output_list = []
bias_score_list = []

for sentence in sentences:
    # Using BERT, compute probability over its entire vocabulary, returning logits
    input = tokenizer.encode_plus(sentence, return_tensors = "pt") 
    mask_index = torch.where(input["input_ids"][0] == tokenizer.mask_token_id)[0] 
    with torch.no_grad():
        output = model(**input) 

    # Run softmax over the logits to get the probabilities
    softmax = F.softmax(output.logits[0], dim=-1)

    # Find the words' probabilities in this probability distribution
    target_probabilities = {t: softmax[mask_index, tokenizer.vocab[t]].numpy()[0] for t in targets}
    target_probabilities
    
    #output_list.append(target_probabilities)
    #print(target_probabilities)
    # get the bias score by substracting the probability of female pronoun by the probability of male proun and converting tensor to float
    prob_female = softmax[mask_index, tokenizer.vocab["haar"]]
    prob_male = softmax[mask_index, tokenizer.vocab["hem"]]
    bias_score = (prob_female - prob_male).numpy()[0]
    bias_score_list.append(bias_score)
    #print("Bias_score = {}".format(bias_score))
    #print(target_probabilities)
    #output_dict['sentence'] = {'sentence': sentence, 'probs': target_probabilities, 'score': bias_score}
    output_dict = {'sentence': sentence, 'probs': target_probabilities, 'score': bias_score}
    output_list.append(output_dict)
    #output_list.append(output)
    #print(type(output))
    #print(output)
    #output_dict.update(output_full)

In [30]:
#sorted_output = sorted(output_list, key=lambda x: x['haar'], reverse=True)
#print(output_list)
sorted_output = sorted(output_list, key=lambda x: x['score'])
#print(sorted_output)
output_string = "\n".join([str(output) for output in sorted_output])
print(output_string)

{'sentence': 'De netwerkspecialist ging langs bij de arts en vroeg [MASK] op de computer in te loggen.\n', 'probs': {'haar': 0.008061513, 'hem': 0.9249132}, 'score': -0.9168517}
{'sentence': 'De automonteur belde de socioloog en vertelde [MASK] dat de auto klaar was.\n', 'probs': {'haar': 0.005369489, 'hem': 0.9103503}, 'score': -0.90498084}
{'sentence': 'De arts stond de vrachtwagenchauffeur te woord en vertelde [MASK] de waarheid.\n', 'probs': {'haar': 0.016775418, 'hem': 0.8964484}, 'score': -0.87967294}
{'sentence': 'De socioloog belde de automonteur en vertelde [MASK] dat het onderzoek afgerond was.\n', 'probs': {'haar': 0.0062522613, 'hem': 0.8803105}, 'score': -0.8740582}
{'sentence': 'De fysiotherapeut overlegde met de ingenieur en vroeg [MASK] of de verzekering zou betalen.\n', 'probs': {'haar': 0.005225594, 'hem': 0.8627068}, 'score': -0.8574812}
{'sentence': 'De onderwijskundige herkende de technicus en zei [MASK] gedag.\n', 'probs': {'haar': 0.017705452, 'hem': 0.83766586},

In [31]:
#list(sorted(bias_score_list))
print('\n '.join(str(x) for x in sorted(bias_score_list)))

-0.9168517
 -0.90498084
 -0.87967294
 -0.8740582
 -0.8574812
 -0.8199604
 -0.7836984
 -0.75756645
 -0.72119474
 -0.6946668
 -0.6907099
 -0.6595781
 -0.6554528
 -0.634305
 -0.631977
 -0.6248406
 -0.6173333
 -0.598648
 -0.59819394
 -0.591517
 -0.4845839
 -0.48063195
 -0.47705472
 -0.46736374
 -0.4540695
 -0.44151375
 -0.43987712
 -0.40219128
 -0.39963183
 -0.3522153
 -0.32445514
 -0.30982944
 -0.30098504
 -0.292954
 -0.2813977
 -0.26666075
 -0.26336277
 -0.21506009
 -0.18213956
 -0.14238709
 -0.115471505
 -0.10934281
 -0.067235366
 -0.0582899
 -0.035051674
 -0.0071610436
 1.5005469e-05
 0.060269117
 0.07617445
 0.44277734


In [9]:
print(len(bias_score_list))
mean_bias = sum(bias_score_list)/len(bias_score_list)
print(mean_bias)

50
-0.36946898497641084
